In [24]:
import re,math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.stats import pearsonr
from scipy.optimize import leastsq


data1 = pd.read_csv(r'细胞间相似性（皮尔森）.csv',engine='python')

In [25]:
# 细胞系-细胞系
high_data = data1.set_index(data1.loc[:,"Unnamed: 0"]).drop(["Unnamed: 0"], axis=1)

In [27]:
# 药物对-细胞系
data2 = pd.read_csv('label_gai.csv')

In [28]:
# 选取细胞系名称
columns = list(data2.columns[1:])

In [29]:
# 删去对应行后的字典
dict_high_lines = {}
count = 0
for i in high_data.index:
    y = list(high_data.loc[:,i])
    y.pop(count)
    count += 1  
    dict_high_lines[i] = y

In [30]:
coum=13

In [31]:
# 分离x y数据构造字典
dict_columns = {}
for i in columns: 
    
    # b
    y = data2.loc[:,i]
    
    # A
    temp = []
    for j in columns:
        if j != i:
            temp.append(j)
    m = data2.loc[:, temp[0]]
    tp = [m]
    for j in temp[1:]:
        tp.append(data2.loc[:, j])
    x = pd.DataFrame(tp).T
    
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    max_l = df.index[-coum:]
    tp = []
    for j in max_l:
        tp.append(data2.loc[:, j])
    max_5x = pd.DataFrame(tp).T
    
    # 存储
    data = {
        "x": x,
        "y": y,
        "max_relevance": max_l,
        "max_5x": max_5x
    }
    dict_columns[i] = data

In [32]:
dict_columns_split = {}
for i in dict_columns: # 第几列
    count = 0 # 第几行 
    for j in list(dict_columns[i]['y'].index):
        dict_columns_split[f'{i},{j}']={}
        dict_columns_split[f'{i},{j}']['y_line']=dict_columns[i]["y"][count]
        dict_columns_split[f'{i},{j}']['max_x_line']=dict_columns[i]["max_5x"].iloc[count,:].values
        count += 1
# 第几个 第几行
len(dict_columns_split)

20405

In [33]:
# 删去对应行后的字典
dict_high_lines = {}
count = 0
for i in high_data.index:
    y = list(high_data.loc[:,i])
    y.pop(count)
    count += 1
    dict_high_lines[i] = y

    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    
    max_l = df.index[-coum:] 
    tp = []
    for j in max_l:
        tp.append(high_data.loc[j, :])
    max_5y = list(pd.DataFrame(tp).T.loc[[i]].values)[0]
    
    data = {
        "max_relevance": max_l,
        "y": y,
        "high_5": max_5y
    }
    dict_high_lines[i] = data

In [34]:
# 筛选每个细胞系前5细胞系得分平均值大于0.92的细胞系
end_score = []
name_dict_high_lines = []
for elem in dict_high_lines:
    end_score.append(dict_high_lines[elem]['high_5'].mean())
    name_dict_high_lines.append(elem)
dict_high_lines_temp = {}
for score in end_score:
    if score >= 0.90:
        dict_high_lines_temp[name_dict_high_lines[end_score.index(score)]] = dict_high_lines[name_dict_high_lines[end_score.index(score)]]
dict_high_lines = dict_high_lines_temp.copy()

In [36]:
dict_high_lines_split = {}
for i in dict_high_lines:
    count = 0
    for j in dict_high_lines[i]['max_relevance']:
        dict_high_lines_split[f'{i},{j}']=dict_high_lines[i]['high_5'][count]
        count += 1
# 相似性
dict_high_lines_split

{'A2058_SKIN ,RPMI7951_SKIN ': 0.9107469992357274,
 'A2058_SKIN ,COLO320_LARGE_INTESTINE ': 0.9130481917910079,
 'A2058_SKIN ,MSTO211H_PLEURA ': 0.9137998398049872,
 'A2058_SKIN ,HT29_LARGE_INTESTINE ': 0.9148348509696906,
 'A2058_SKIN ,NCIH1650_LUNG ': 0.9225936507954852,
 'A2058_SKIN ,SW620_LARGE_INTESTINE ': 0.9257557082760705,
 'A2058_SKIN ,CAOV3_OVARY ': 0.9305942207588987,
 'A2058_SKIN ,A2780_OVARY ': 0.9331314887615868,
 'A2058_SKIN ,MDAMB436_BREAST ': 0.9343164083696928,
 'A2058_SKIN ,HT144_SKIN ': 0.9395255879946088,
 'A2058_SKIN ,SKMEL30_SKIN ': 0.9408040042735772,
 'A2058_SKIN ,UACC62_SKIN ': 0.9442572894758484,
 'A2058_SKIN ,NCIH460_LUNG ': 0.9480374602626932,
 'A2780_OVARY ,HT29_LARGE_INTESTINE ': 0.9066952601256056,
 'A2780_OVARY ,A427_LUNG ': 0.9114163262248176,
 'A2780_OVARY ,OVCAR4_OVARY ': 0.9131079131393179,
 'A2780_OVARY ,MDAMB436_BREAST ': 0.9141468190716704,
 'A2780_OVARY ,SKMEL30_SKIN ': 0.919813875991606,
 'A2780_OVARY ,NCIH460_LUNG ': 0.9229220482909352,
 'A278

In [39]:
import threading
import queue
from tqdm import tqdm

q = queue.Queue()
result_q = queue.Queue()

for elem in range(583):
    q.put(elem)


def temp_f():
    while True:
        if q.qsize() != 0:
            elem = q.get()
            mse_split = []
            for alp in np.linspace(0.0001, 0.05, 500):
                r1 = []
                ct = 0
                for linename in dict_high_lines:
                    count = 0
                    pp = []
                    for i in dict_high_lines_split:
                        if count >= 13 * ct and count < (13 * ct + 13):
                            pp.append(np.exp(-pow(1 - dict_high_lines_split[i], 2) / (2 * pow(alp, 2))))
                        count += 1
                    ct += 1
                    r1.append(np.sum(pp * dict_columns_split[f"{linename},{elem}"]['max_x_line']) -
                              dict_columns_split[f"{linename},{elem}"]['y_line'])
                mse_split.append(np.mean(np.square(r1)))
            result_q.put((np.min(mse_split), lin[mse_split.index(np.min(mse_split))]))
            del mse_split, r1, ct, count, pp, elem
        else:
            break




threads = []
for i in tqdm(range(100)):
    t = threading.Thread(target=temp_f)
    threads.append(t)
    t.start()

print('主程序运行中...')

# 等待所有线程任务结束。
for t in threads:
    t.join()

print("所有线程任务完成")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [05:17<00:00,  3.18s/it]


主程序运行中...
所有线程任务完成


In [41]:
result_mse = []
result_alp = []
for i in range(result_q.qsize()):
    j = result_q.get()
    result_mse.append(j[0])
    result_alp.append(j[1])

In [42]:
np.sum(result_mse)/583

396.96455391577234

In [18]:
result_alp

[0.0001,
 0.0308,
 0.0055000000000000005,
 0.0131,
 0.029400000000000003,
 0.0269,
 0.0001,
 0.0001,
 0.0001,
 0.031900000000000005,
 0.0291,
 0.0293,
 0.029500000000000002,
 0.029500000000000002,
 0.0238,
 0.028300000000000002,
 0.0001,
 0.0262,
 0.0001,
 0.033,
 0.0308,
 0.0308,
 0.0292,
 0.0298,
 0.0001,
 0.026600000000000002,
 0.0286,
 0.0292,
 0.0123,
 0.0216,
 0.0292,
 0.030600000000000002,
 0.0303,
 0.025,
 0.0291,
 0.0001,
 0.0245,
 0.0284,
 0.0213,
 0.0292,
 0.0297,
 0.0311,
 0.0303,
 0.0288,
 0.025,
 0.0001,
 0.029,
 0.023200000000000002,
 0.0275,
 0.021,
 0.016,
 0.005,
 0.0292,
 0.0282,
 0.0242,
 0.0224,
 0.0273,
 0.0281,
 0.0104,
 0.0156,
 0.0142,
 0.0001,
 0.0178,
 0.015700000000000002,
 0.022,
 0.027800000000000002,
 0.029,
 0.0292,
 0.0001,
 0.029,
 0.0001,
 0.0127,
 0.0298,
 0.0206,
 0.0282,
 0.023,
 0.0261,
 0.029400000000000003,
 0.0274,
 0.0262,
 0.0307,
 0.0231,
 0.0308,
 0.0273,
 0.0001,
 0.0262,
 0.0263,
 0.0001,
 0.0298,
 0.0233,
 0.020300000000000002,
 0.0263,


In [19]:
import pickle

with open('细胞系alpha.pkl', 'wb') as file:
    pickle.dump(result_alp, file) # 权重

In [20]:
dict_columns_split

{'A2058_SKIN ,0': {'y_line': -9.51,
  'max_x_line': array([ -8.65 ,   5.36 ,  -6.96 ,   4.7  ,  11.2  ,   5.1  , -16.4  ,
           2.6  ,   4.87 ,   5.79 ,   9.6  ,   3.88 ,  -0.127])},
 'A2058_SKIN ,1': {'y_line': -13.2,
  'max_x_line': array([-22.5  ,  16.3  ,   9.99 , -20.3  ,   0.678,  13.8  , -15.6  ,
          -4.03 ,  -3.48 ,   9.68 ,   4.7  , -10.7  ,   6.71 ])},
 'A2058_SKIN ,2': {'y_line': 26.4,
  'max_x_line': array([-16.7 ,  10.3 , -10.8 ,  14.5 ,   1.52,  10.5 ,  17.7 ,  14.5 ,
           7.15,   8.4 ,  10.2 ,  18.8 ,   7.37])},
 'A2058_SKIN ,3': {'y_line': 4.33,
  'max_x_line': array([ -5.69,   4.81,   1.11,  -2.54,  -2.11,  -4.57, -14.3 ,  -8.16,
           5.35,  -3.48, -96.6 ,   3.68,  -7.42])},
 'A2058_SKIN ,4': {'y_line': 45.5,
  'max_x_line': array([10.3 , 22.7 , -6.04, 37.3 , 23.5 , 18.4 , 20.1 , 22.5 , 14.5 ,
         23.4 , 26.3 , 34.7 , 29.  ])},
 'A2058_SKIN ,5': {'y_line': -5.01,
  'max_x_line': array([ -2.72,  21.2 ,  40.  ,   1.02,   5.69,   8.65, -15.3 , 

In [21]:
def dictW(alpha):
    # 运算构造参数W字典 
    dict_W = {}
    for elem in dict_high_lines:
        r = np.array(dict_high_lines[elem]["high_5"])
        W1= pow((1-r),2)
        W = np.exp(-W1/(2*pow(alpha,2)))
        dict_W[elem] = W
    return dict_W

def dictr(dict_W):
    # 均方误差字典 （MSE=
    # r = max(|A×W - b|)
    dict_r = {}
    for elem,w in zip(dict_columns, dict_W):
        A = np.array(dict_columns[elem]["max_5x"].values) # x----->max_5x 取列
        b = np.array(dict_columns[elem]["y"].values) # 那一列
        W = dict_W[w]
        temp1 = np.dot(A,W) - b
        temp2 = np.square(temp1)
        temp3 = np.mean(temp2)
        dict_r[w] = temp3
    return dict_r

def dirlist(dict_r):
    #计算误差平方和    
    dir_list = []
    for elem in dict_r:
        dir_list.append(np.sqrt(dict_r[elem]))
    return dir_list

In [22]:
dict_W = dictW(alpha)

dict_rr = dictr_split(dict_W)
dict_rr

NameError: name 'alpha' is not defined

In [ ]:
dict_rr = dictr_split(dict_W)

dir_list=dirlist(dict_rr)

temp=np.mean(dir_list)

In [ ]:
dev=10000000
lin=np.linspace(0.001,0.1,100)
for alpha in lin:
    
    dict_W = dictW(alpha)
    
    dict_rr = dictr_split(dict_W)
 
    dir_list=dirlist(dict_rr)

    temp=np.mean(dir_list)
                      
    #误差判断
    if(temp>=dev):
        continue
    else:
        dev=temp
        al=alpha

alpha=al
dict_W = dictW(alpha)
dict_rr = dictr_split(dict_W)

# 存储最终数据
import pickle

with open('细胞系.pkl', 'wb') as file:
    pickle.dump(dict_W, file) # 权重
    pickle.dump(dict_columns, file) # 细胞系得分所有数据
    pickle.dump(dict_high_lines, file) # 细胞系得分高的数据

In [ ]:
# MSE
np.array(list(dict_rr.values())).mean()

In [ ]:
#5  456.1597608045107
#10 452.7681291721725
#11 452.402461862494
#13 452.12224622781355
#14 452.1240519728004
#15 452.1913027183096